<a href="https://colab.research.google.com/github/Lasitha-Jayawardana/IDS/blob/main/EM_Kmeans_UNSW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import numpy as np
import pandas as pd
import itertools
from scipy import linalg
import matplotlib.pyplot as plt
import matplotlib as mpl
from sklearn.decomposition import PCA
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelEncoder
from sklearn.compose import make_column_transformer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV
import math
from sklearn.cluster import KMeans
# To change scientific numbers to float
np.set_printoptions(formatter={'float_kind':'{:f}'.format})
from sklearn.model_selection import GridSearchCV, cross_val_score,train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, classification_report, confusion_matrix,f1_score
from sklearn import preprocessing 
from sklearn.utils import shuffle
from sklearn.mixture import GaussianMixture
from sklearn.metrics import make_scorer
from sklearn.pipeline import Pipeline
from sklearn.metrics import roc_auc_score

In [3]:
def loadData():
  global df_train
  global df_test
  
  df_train = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IDS Project//UNSW_NB15_training-set.csv')
  df_test = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/IDS Project//UNSW_NB15_testing-set.csv')

  for col in ['proto','service','state']:
    df_train[col] = df_train[col].astype('category')
    df_test[col] = df_test[col].astype('category')

  df_train=df_train.drop(columns='id')
  df_test=df_test.drop(columns='id')
  df_test.drop(df_test[df_test['state'] == 'ACC'].index, inplace = True)
  df_test.drop(df_test[df_test['state'] == 'CLO'].index, inplace = True)
  
  df_train = shuffle(df_train)
  df_test = shuffle(df_test)
  df_test.reset_index(drop=True,inplace=True) 
  df_train.reset_index(drop=True,inplace=True) 

In [4]:
def balanceClass():
  global df_train
  global df_test
  # Class count
  count_class_0, count_class_1 = df_train['label'].value_counts()

  # Divide by class
  df_class_1= df_train[df_train['label'] == 0]
  df_class_0 = df_train[df_train['label'] == 1]
      
  df_class_0_under = df_class_0.sample(count_class_1)
  df_train = pd.concat([df_class_0_under, df_class_1], axis=0)
  df_train.reset_index(drop=True,inplace=True)
  df_train.groupby('label')['label'].count()

In [5]:
def addOtherLabel():
  global df_train
  global df_test
  label1 = 'other_proto'
  others2 = df_train['proto'].value_counts().index[7:]
  df_train['proto'] = df_train['proto'].cat.add_categories([label1])
  df_train['proto'] = df_train['proto'].replace(others2, label1)
  
  label2 = 'other_state'
  others3 = df_train['state'].value_counts().index[4:]
  df_train['state'] = df_train['state'].cat.add_categories([label2])
  df_train['state'] = df_train['state'].replace(others3, label2)
  
  df_test.drop(df_test[df_test['state'] == 'ACC'].index, inplace = True)
  df_test.drop(df_test[df_test['state'] == 'CLO'].index, inplace = True)
   
  others4 = df_test['proto'].value_counts().index[7:]
  df_test['proto'] = df_test['proto'].cat.add_categories([label1])
  df_test['proto'] = df_test['proto'].replace(others4, label1)
  
   
  others5 = df_test['state'].value_counts().index[4:]
  df_test['state'] = df_test['state'].cat.add_categories([label2])
  df_test['state'] = df_test['state'].replace(others5, label2)
  
  
  
#addOtherLabel()

In [6]:
def removeDuplicate():
  print('Duplicates train:',df_train.duplicated().sum())
  df_train.drop_duplicates(keep='first',inplace=True)
  print(df_train.duplicated().sum())

  print('Duplicates test:',df_test.duplicated().sum())
  df_test.drop_duplicates(keep='first',inplace=True)
  print(df_test.duplicated().sum())

  df_train.reset_index(drop=True,inplace=True)
  df_test.reset_index(drop=True,inplace=True)

In [7]:
def splitData():
  global X_train
  global Y_train
  global X_test
  global Y_test
  global df_train
  global df_test
  
  limit1 = df_train.shape[1]-1
      
  X_train = df_train.iloc[:,0:limit1-1] # train set features
  Y_train = df_train.iloc[:,limit1]
      
  limit2 = df_test.shape[1]-1
      
  X_test = df_test.iloc[:,0:limit2-1] # test set features
  Y_test = df_test.iloc[:,limit2]
  X_train.reset_index(drop=True,inplace=True)
  Y_train.reset_index(drop=True,inplace=True)
  X_test.reset_index(drop=True,inplace=True)
  Y_test.reset_index(drop=True,inplace=True)

In [8]:
def labelEncoding(X_train,X_test):
  label_encoder = preprocessing.LabelEncoder() 
  
# Encode labels in column 'species'. 
  X_train['proto']= label_encoder.fit_transform(X_train['proto' ]) 
  X_test['proto']= label_encoder.transform(X_test['proto' ]) 
  X_train['state']= label_encoder.fit_transform(X_train['state' ]) 
  X_test['state']= label_encoder.transform(X_test['state' ]) 
  X_train['service']= label_encoder.fit_transform(X_train['service' ]) 
  X_test['service']= label_encoder.transform(X_test['service' ]) 
  return X_train,X_test

#X_train,X_test = labelEncoding(X_train,X_test)


In [9]:
def one_hot_encode(X_train,X_test):
    
    categorical_cols = ['proto','service','state']
    # Training dataset one hot encoding
    ohe = OneHotEncoder(handle_unknown = 'ignore')
    ohe.fit(X_train[categorical_cols])
    array_hot_encoded1 = ohe.transform(X_train[categorical_cols]).toarray()

    data_hot_encoded1 = pd.DataFrame(array_hot_encoded1, index=X_train.index,columns=ohe.get_feature_names(categorical_cols))
    X_train = X_train.drop(columns=categorical_cols)
    X_train = pd.concat([data_hot_encoded1,X_train], axis=1)
    
    print('X_train shape :',X_train.shape)
    
    # Test dataset one hot encoding
    array_hot_encoded2 = ohe.transform(X_test[categorical_cols]).toarray()
    data_hot_encoded2 = pd.DataFrame(array_hot_encoded2, index=X_test.index,columns=ohe.get_feature_names(categorical_cols))
    X_test = X_test.drop(columns=categorical_cols)
    X_test = pd.concat([data_hot_encoded2,X_test], axis=1)
    
    print('X_test shape :',X_test.shape)
    X_train = pd.DataFrame(X_train)
    X_test = pd.DataFrame(X_test)

    Y_train.reset_index(drop=True,inplace=True)
    Y_test.reset_index(drop=True,inplace=True)
    return X_train, X_test

#X_train, X_test = one_hot_encode(X_train,X_test)

In [10]:
def scaling(X_train,X_test,scaler):
  
  X_train = pd.DataFrame(scaler.fit_transform(X_train),columns=X_train.columns)
  X_test = pd.DataFrame(scaler.transform(X_test),columns=X_test.columns)
  return X_train,X_test

#X_train, X_test = scaling(X_train,X_test,StandardScaler())

In [11]:
def runPCA(X_train,X_test,kPCA):
  pca = PCA(n_components=kPCA, random_state = 453)
  X_rtrain = pca.fit(X_train).transform(X_train)
  X_rtest  = pca.transform(X_test)
  return X_rtrain,X_rtest

In [12]:

def searchGmm(X):
  lowest_bic = np.infty
  bic = []
  n_components_range = range(8, 20)
  cv_types = ['spherical', 'tied', 'diag', 'full']
  for cv_type in cv_types:
      for n_components in n_components_range:
          # Fit a Gaussian mixture with EM
          print ("K = ",n_components)
          gmm = GaussianMixture(n_components=n_components,covariance_type=cv_type,init_params='kmeans',random_state=12,verbose=0,verbose_interval=10,max_iter=1000)

          gmm.fit(X)
          bic.append(gmm.bic(X))
          if bic[-1] < lowest_bic:
              lowest_bic = bic[-1]
              best_gmm = gmm

  bic = np.array(bic)
  color_iter = itertools.cycle(['navy', 'turquoise', 'cornflowerblue',
                                'darkorange'])
  clf = best_gmm
  bars = []
  print(lowest_bic)
  # Plot the BIC scores
  plt.figure(figsize=(8, 6))
  spl = plt.subplot(2, 1, 1)
  for i, (cv_type, color) in enumerate(zip(cv_types, color_iter)):
      xpos = np.array(n_components_range) + .2 * (i - 2)
      bars.append(plt.bar(xpos, bic[i * len(n_components_range):
                                    (i + 1) * len(n_components_range)],
                          width=.2, color=color))
  plt.xticks(n_components_range)
  plt.ylim([bic.min() * 1.01 - .01 * bic.max(), bic.max()])
  plt.title('BIC score per model')
  xpos = np.mod(bic.argmin(), len(n_components_range)) + .65 +\
      .2 * np.floor(bic.argmin() / len(n_components_range))
  plt.text(xpos, bic.min() * 0.97 + .03 * bic.max(), '*', fontsize=14)
  spl.set_xlabel('Number of components')
  spl.legend([b[0] for b in bars], cv_types)


In [13]:
def searchKmeans():
 

  # Running K means with multible Ks
  best_seed = None
  

  X_value = X_train
  min_inertia = []
  #X_value = newdata.values
  seeds = [0, 20000, 40000, 60000, 80000, 100000, 120000]
  for seed in seeds:
    inertia = []
    no_of_clusters = range(2,10)
    print('\n\n seed= {} \n'.format(seed))
    for f in no_of_clusters:
        kmeans = KMeans(n_clusters=f, init='k-means++',random_state=seed)
        kmeans = kmeans.fit(X_value)
        
        u = kmeans.inertia_
        inertia.append(u)
        print("The innertia for :", f, "Clusters is:", u) 
        # if current measurement of heterogeneity is lower than previously seen,
        # update the minimum record of heterogeneity.
    if len(min_inertia) == 0 :
        min_inertia = inertia
        
    if min(inertia,default=0) < min(min_inertia,default=0):
        min_inertia = inertia
        best_seed = seed
    
    # Creating the scree plot for Intertia - elbow method
  fig, (ax1) = plt.subplots(1, figsize=(16,6))
  xx = np.arange(len(no_of_clusters))
  ax1.plot(xx, min_inertia,linewidth=4)
  ax1.set_xticks(xx)
  ax1.set_xticklabels(no_of_clusters, rotation='vertical')
  plt.xlabel('Number of clusters')
  plt.ylabel('Inertia Score')
  plt.title('Inertia Plot per k for Best Seed {}'.format(best_seed))

  # between-a-point-and-a-line-in-2-d/
  def calc_distance(x1, y1, a, b, c):
    d = abs((a * x1 + b * y1 + c)) / (math.sqrt(a * a + b * b))
    return d

  a = inertia[0] - inertia[-1]
  b = no_of_clusters[-1] - no_of_clusters[0]
  c1 = no_of_clusters[0] * inertia[-1]
  c2 = no_of_clusters[-1] * inertia[0]
  c = c1 - c2

  r = no_of_clusters[-1]-1
  r
  distance_of_points_from_line = []

  for k in range(int(r)):
    distance_of_points_from_line.append(
        calc_distance(no_of_clusters[k], inertia[k], a, b, c))
    
  plt.plot(no_of_clusters, distance_of_points_from_line)
  K =no_of_clusters[distance_of_points_from_line.index(max(distance_of_points_from_line))]
  print("Best K value : {}".format(K))

  return  K,best_seed 

In [14]:
def XtoN():
  global X_trainN
  global X_testN
  global kmeans
  X_trainN = X_train
  X_testN = X_test
  

In [15]:
# Running K means with multible seeds
 
def searchSeed(X_train,no_of_clusters):
  global kmeans
  best_seed = None


  min_inertia=0.0
  
  seeds = [0,9500,10000,10500,11000,15000,18000 , 20000, 40000, 60000, 80000,90000, 120000]
  for seed in seeds:

    
    #print('\n seed= {} \n'.format(seed))
        
    kmeans = KMeans(n_clusters=no_of_clusters, init='k-means++',random_state=seed)
    kmeans = kmeans.fit(X_train)
    
    inertia = kmeans.inertia_
    #print("The innertia for : 2 Clusters is:", inertia) 
    # if current measurement of heterogeneity is lower than previously seen,
    # update the minimum record of heterogeneity.
    if min_inertia == 0 :
        min_inertia = inertia
        best_seed = seed
    if inertia < min_inertia:
        min_inertia = inertia
        best_seed = seed
        

  print("\nMin inertia : ", min_inertia)
  print("Best Seed : ",best_seed)
  return best_seed
 

In [16]:
def runKmeans(k,best_seed):
  global kmeans
  # Running K means on K clusters
  kmeans = KMeans(n_clusters=k,init='k-means++', random_state=best_seed)
  kmeans = kmeans.fit(X_trainN)

In [17]:
def binaryaccuracyScore():
  label0 = 1- kmeans.labels_
  label1 = kmeans.labels_
  test_label1 = kmeans.predict(X_testN)
  test_label0 = 1 - test_label1

  a = accuracy_score(Y_train,label0)*100
  aa = accuracy_score(Y_train,label1)*100
  if (a>= aa):
    l = test_label0
    print("Accuracy Train: ",a)
  else:
    l = test_label1
    print("Accuracy Train: ",aa)

  print("Accuracy Test: ",accuracy_score(Y_test,l)*100)

  print("\nTest set precision : {:.4f}".format(precision_score(Y_test, l)))
  print("Test set recall    : {:.4f}".format(recall_score(Y_test, l)))
  print("Test set F1-score  : {:.4f}".format(f1_score(Y_test, l)))
  
  cm = confusion_matrix(Y_test,l,normalize= 'true')
  print('True Positives  : {:.2f}'.format(cm[1][1]))
  print('True Negatives  : {:.2f}'.format(cm[0][0]))
  print('False Positives : {:.2f}'.format(cm[0][1]))
  print('False Negatives : {:.2f}'.format(cm[1][0]))

In [27]:
def oneVsAllScore(n,Trainlabel,Testlabel): 

  def oneVsAll(number):
    if (number ==nn):
      return 0
    else:
      return 1
  m=[]
  
  for i in range(0, n):
    nn=i
    m.append(accuracy_score(Y_train,np.array(list(map(oneVsAll, Trainlabel)))))
    print(i,"  ",accuracy_score(Y_train,np.array(list(map(oneVsAll, Trainlabel)))))

  print("Accuracy Train: ",max(m)*100)
  index =m.index(max(m))
  nn=index
  #print(nn)
  a=np.array(list(map(oneVsAll, Testlabel)))
  print("Accuracy Test: ",accuracy_score(Y_test,a)*100)
  print("Test set precision : {:.4f}".format(precision_score(Y_test, a)))
  print("Test set recall    : {:.4f}".format(recall_score(Y_test, a)))
  print("Test set F1-score  : {:.4f}".format(f1_score(Y_test, a)))
  print("Test set Roc       : {:.4f}".format(roc_auc_score(Y_test, a)))
  cm = confusion_matrix(Y_test,a,normalize= 'true')
  print('True Positives  : {:.2f}'.format(cm[1][1]))
  print('True Negatives  : {:.2f}'.format(cm[0][0]))
  print('False Positives : {:.2f}'.format(cm[0][1]))
  print('False Negatives : {:.2f}'.format(cm[1][0]))
 

In [19]:
def mapMethodScore(n):
  
  l0=[]
  Trainlabel = kmeans.labels_
  Testlabel = kmeans.predict(X_testN)
  def lmap(num):
    if (num in l0):
      return 0
    else:
      return 1
  
  for i in range(0, n):

    #print("Cluster ", i)
    c = Y_train[Trainlabel==i]
    if (len(c[c == 1])<len(c[c == 0])):
      l0.append(i)
      #print("add to normal ",i)
    #print("intrusion : ",len(c[c == 1]))
    #print("normal : ",len(c[c == 0]))

  ltrain = np.array(list(map(lmap , Trainlabel)))
  print("Accuracy Train: ",accuracy_score(Y_train,ltrain)*100)

  ltest = np.array(list(map(lmap , Testlabel)))

  print("Accuracy Test: ",accuracy_score(Y_test,ltest)*100)
  print("Test set precision : {:.4f}".format(precision_score(Y_test,ltest)))
  print("Test set recall    : {:.4f}".format(recall_score(Y_test,ltest)))
  print("Test set F1-score  : {:.4f}".format(f1_score(Y_test,ltest)))
  cm = confusion_matrix(Y_test,ltest,normalize= 'true')
  print('True Positives  : {:.2f}'.format(cm[1][1]))
  print('True Negatives  : {:.2f}'.format(cm[0][0]))
  print('False Positives : {:.2f}'.format(cm[0][1]))
  print('False Negatives : {:.2f}'.format(cm[1][0]))



In [19]:
y_true = [1,0]
y_pred = [1,1]
print(recall_score(y_true, y_pred))
print(precision_score(y_true, y_pred))
print(roc_auc_score(y_true, y_pred))
print(f1_score(y_true, y_pred))
cm = confusion_matrix(y_true,y_pred,normalize= 'true')
print('True Positives  : {:.2f}'.format(cm[1][1]))
print('True Negatives  : {:.2f}'.format(cm[0][0]))
print('False Positives : {:.2f}'.format(cm[0][1]))
print('False Negatives : {:.2f}'.format(cm[1][0]))

1.0
0.5
0.5
0.6666666666666666
True Positives  : 1.00
True Negatives  : 0.00
False Positives : 1.00
False Negatives : 0.00


In [21]:
#.......................................................................Kmeans.......................................................
trainScore = True
cvCount = 5

last=1 #test=0
i_index=-1
myScore=[]
iCount=0
fitNum=0


def acc_score(actual,prediction,method):
  global last
  global i_index
  global myScore
  global iCount
  global fitNum
 

  m=[[],[],[],[],[]]
  cls = -1
  
  
  def oneVsAll(num):
    if (num ==cls):
      return 0
    else:
      return 1

  if (last == 0 and trainScore):
     #trainset
    print("in train")
    
    myScore.clear()
    cls = i_index
    #print("Index--------",i_index)
    oneVtest =np.array(list(map(oneVsAll, prediction)))
    myScore.append(accuracy_score(actual,oneVtest)*100)
    myScore.append(precision_score(actual,oneVtest))
    myScore.append(recall_score(actual,oneVtest))
    myScore.append(f1_score(actual,oneVtest))
    myScore.append(roc_auc_score(actual,oneVtest))

    last = 1
    
  else:
   #testset
    print("in test")
    lena = len(np.unique(actual))
    lenp = len(np.unique(prediction))
    if (lena>lenp):
      n = lena
    else:
      n= lenp   
    

    if (iCount%cvCount == 0):
      
      fitNum =fitNum + 1
      print("-----------------------------------------------------------",fitNum,"--------------------------------------------------------")
      
       
    
    myScore.clear()
    
    for i in range(0, n):
      cls=i
      oneVall=np.array(list(map(oneVsAll, prediction)))
      #print(len(np.unique(actual)))
      
      m[0].append(accuracy_score(actual,oneVall)*100)
      m[1].append(precision_score(actual,oneVall))
      m[2].append(recall_score(actual,oneVall))
      m[3].append(f1_score(actual,oneVall))
      m[4].append(roc_auc_score(actual,oneVall))
      
    
    i_index = m[0].index(max(m[0]))  
    #print("Index--------",i_index)
    myScore.append(m[0][i_index])
    myScore.append(m[1][i_index])
    myScore.append(m[2][i_index])
    myScore.append(m[3][i_index])
    myScore.append(m[4][i_index])

    last = 0
    #print("accurcy max---",myScore[0])
  return myScore[0]
  


def prec_score(estimator,x,y):
  return myScore[1]

def rec_score(estimator,x,y):
  return myScore[2]

def f1_s_score(estimator,x,y):
  return myScore[3]

def roc_score(estimator,x,y):
  return myScore[4]




loadData()


#addOtherLabel()
removeDuplicate()
#balanceClass()
print(df_train.groupby('label')['label'].count())
splitData()

X_train, X_test = one_hot_encode(X_train,X_test)



scalers_to_test = [StandardScaler()]#
 
params = {'scaler': scalers_to_test,
          'reduce_dim': [None,PCA(n_components=2),PCA(n_components=3)],
          'regressor__n_clusters': [10],
          'regressor__random_state':[2000,100000]}
          
"""
....................................or (both result same)
params = [{'scaler': scalers_to_test,
          'reduce_dim': [PCA()],
          'reduce_dim__n_components': [5,20,40,55],
          'regressor__n_clusters': [2,5],
          'regressor__random_state':[0,2000,10000,50000,100000]}
          ,
          {
          'scaler': scalers_to_test,
          'reduce_dim': [None],
          'regressor__n_clusters': [2,5],
          'regressor__random_state':[0,2000,10000,50000,100000]    
          }]

"""
pipe = Pipeline(steps=[('scaler', StandardScaler()),('reduce_dim', PCA()),('regressor', KMeans())],verbose=2)


scoring = {'accuracy': make_scorer(acc_score,greater_is_better=True,method='acc'),
           'prec': prec_score,
           'rec': rec_score,
           'f1_s': f1_s_score,
            'roc': roc_score}          

grid = GridSearchCV(pipe, cv=cvCount, scoring=scoring,param_grid= params,return_train_score=trainScore,refit='accuracy',n_jobs=1, verbose=2)#

grid.fit(X_train,Y_train)

cvResult = pd.DataFrame(grid.cv_results_)
#rmtree(cachedir)

Duplicates train: 67601
0
Duplicates test: 26387
0
label
0    51890
1    55850
Name: label, dtype: int64
X_train shape : (107740, 194)
X_test shape : (55940, 194)
Fitting 5 folds for each of 6 candidates, totalling 30 fits
[CV] reduce_dim=None, regressor__n_clusters=10, regressor__random_state=2000, scaler=StandardScaler(copy=True, with_mean=True, with_std=True) 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.4s
[Pipeline] ........ (step 2 of 3) Processing reduce_dim, total=   0.0s
[Pipeline] ......... (step 3 of 3) Processing regressor, total=  13.0s
in test
----------------------------------------------------------- 1 --------------------------------------------------------
in train
[CV]  reduce_dim=None, regressor__n_clusters=10, regressor__random_state=2000, scaler=StandardScaler(copy=True, with_mean=True, with_std=True), total=  14.5s
[CV] reduce_dim=None, regressor__n_clusters=10, regressor__random_state=2000, scaler=StandardScaler(copy=True, with_mean=True, with_std=True) 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:   15.1s remaining:    0.0s


[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.4s
[Pipeline] ........ (step 2 of 3) Processing reduce_dim, total=   0.0s
[Pipeline] ......... (step 3 of 3) Processing regressor, total=  12.5s
in test
----------------------------------------------------------- 2 --------------------------------------------------------
in train
[CV]  reduce_dim=None, regressor__n_clusters=10, regressor__random_state=2000, scaler=StandardScaler(copy=True, with_mean=True, with_std=True), total=  13.7s
[CV] reduce_dim=None, regressor__n_clusters=10, regressor__random_state=2000, scaler=StandardScaler(copy=True, with_mean=True, with_std=True) 
[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.4s
[Pipeline] ........ (step 2 of 3) Processing reduce_dim, total=   0.0s
[Pipeline] ......... (step 3 of 3) Processing regressor, total=  12.3s
in test
----------------------------------------------------------- 3 --------------------------------------------------------
in train
[

[Parallel(n_jobs=1)]: Done  30 out of  30 | elapsed:  4.5min finished


[Pipeline] ............ (step 1 of 3) Processing scaler, total=   0.5s
[Pipeline] ........ (step 2 of 3) Processing reduce_dim, total=   0.0s
[Pipeline] ......... (step 3 of 3) Processing regressor, total=  14.3s


In [22]:
cc = ['param_reduce_dim','param_regressor__n_clusters','param_regressor__random_state','param_scaler','params','mean_test_accuracy','rank_test_accuracy','mean_test_prec','rank_test_prec','mean_test_rec','rank_test_rec','mean_test_f1_s','rank_test_f1_s','mean_test_roc','rank_test_roc']
s = cvResult[cc]
s.sort_values(['rank_test_accuracy'])

,param_reduce_dim,param_regressor__n_clusters,param_regressor__random_state,param_scaler,params,mean_test_accuracy,rank_test_accuracy,mean_test_prec,rank_test_prec,mean_test_rec,rank_test_rec,mean_test_f1_s,rank_test_f1_s,mean_test_roc,rank_test_roc
0,None,10,2000,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': None, 'regressor__n_clusters': ...",74.849638,1,0.675090,2,0.989124,2,0.801897,1,0.738767,1
4,"PCA(copy=True, iterated_power='auto', n_compon...",10,2000,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': PCA(copy=True, iterated_power='...",74.773529,2,0.676860,1,0.976999,5,0.799454,2,0.737454,2
5,"PCA(copy=True, iterated_power='auto', n_compon...",10,100000,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': PCA(copy=True, iterated_power='...",74.068127,3,0.671424,3,0.972708,6,0.794189,3,0.730448,3
1,None,10,100000,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': None, 'regressor__n_clusters': ...",72.151476,4,0.653728,4,0.989636,1,0.785990,4,0.711875,4
2,"PCA(copy=True, iterated_power='auto', n_compon...",10,2000,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': PCA(copy=True, iterated_power='...",68.118619,5,0.620212,5,0.986023,4,0.760945,5,0.668665,5
3,"PCA(copy=True, iterated_power='auto', n_compon...",10,100000,"StandardScaler(copy=True, with_mean=True, with...","{'reduce_dim': PCA(copy=True, iterated_power='...",68.046222,6,0.619620,6,0.986375,3,0.760600,6,0.667914,6


In [30]:
nu = 5

print(grid.cv_results_['param_reduce_dim'][nu])
print(grid.cv_results_['params'][nu])

PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False)
{'reduce_dim': PCA(copy=True, iterated_power='auto', n_components=3, random_state=None,
    svd_solver='auto', tol=0.0, whiten=False), 'regressor__n_clusters': 10, 'regressor__random_state': 100000, 'scaler': StandardScaler(copy=True, with_mean=True, with_std=True)}


In [31]:
print("Kmean \n\n")
loadData()
#df_train.columns
#print(df_train.groupby('attack_cat')['attack_cat'].count())

#print(df_train.groupby('label')['label'].count())
#addOtherLabel()
removeDuplicate()
#balanceClass()
splitData()


X_train, X_test = one_hot_encode(X_train,X_test)

X_train, X_test = scaling(X_train,X_test,StandardScaler())
X_train,X_test = runPCA(X_train,X_test,3)

kmean= KMeans(n_clusters=10,random_state=100000)
kmean.fit(X_train)

Kmean 


Duplicates train: 67601
0
Duplicates test: 26387
0
X_train shape : (107740, 194)
X_test shape : (55940, 194)


KMeans(algorithm='auto', copy_x=True, init='k-means++', max_iter=300,
       n_clusters=10, n_init=10, n_jobs=None, precompute_distances='auto',
       random_state=100000, tol=0.0001, verbose=0)

In [32]:
predictedTrain = kmean.predict(X_train)
predictedTest= kmean.predict(X_test)


In [33]:
oneVsAllScore(10,predictedTrain, predictedTest)

0    0.756432151475775
1    0.5024132170038983
2    0.3706237237794691
3    0.4162149619454242
4    0.5031557453127901
5    0.518099127529237
6    0.6145721180620011
7    0.41170410246890665
8    0.5241692964544273
9    0.529636161128643
Accuracy Train:  75.64321514757751
Accuracy Test:  58.65212727922774
Test set precision : 0.4841
Test set recall    : 0.9740
Test set F1-score  : 0.6467
Test set Roc       : 0.6571
True Positives  : 0.97
True Negatives  : 0.34
False Positives : 0.66
False Negatives : 0.03


In [ ]:
#.......................................................................Kmeans debug & testing
trainScore = True
cvCount = 5

last=1 #test=0
i_index=-1
myScore=[]
iCount=0
fitNum=0


def acc_score(actual,prediction,method):
  global last
  global i_index
  global myScore
  global iCount
  global fitNum
 

  m=[[],[],[],[],[]]
  cls = -1
  
  
  def oneVsAll(num):
    if (num ==cls):
      return 0
    else:
      return 1

  if (last == 0 and trainScore):
    #trainset
    print("in train")
    pp = pd.DataFrame(actual)
    ppp = pd.DataFrame(prediction)
    print("actual : ",pp.groupby('label')['label'].count())
    print("predicted : ",ppp.groupby(0)[0].count())
    myScore.clear()
    cls = i_index
    #print("Index--------",i_index)
    oneVtest =np.array(list(map(oneVsAll, prediction)))
    myScore.append(accuracy_score(actual,oneVtest)*100)
    myScore.append(precision_score(actual,oneVtest))
    myScore.append(recall_score(actual,oneVtest))
    myScore.append(f1_score(actual,oneVtest))
    myScore.append(roc_auc_score(actual,oneVtest))

    last = 1
    
  else:
   #testset
    print("in test")
    lena = len(np.unique(actual))
    lenp = len(np.unique(prediction))
    if (lena>lenb):
      n = lena
    else:
      n= lenb 
    if (iCount%cvCount == 0):
      
      fitNum =fitNum + 1
      print("-----------------------------------------------------------",fitNum,"--------------------------------------------------------")
      
    pp = pd.DataFrame(actual)
    ppp = pd.DataFrame(prediction)
    print("actual : ",pp.groupby('label')['label'].count())
    print("predicted : ",ppp.groupby(0)[0].count())   
    
    myScore.clear()
    if ((len(np.unique(actual))<2) or (len(np.unique(prediction))<2)):
      print("############################################################################################################################")
    for i in range(0, n):
      cls=i
      oneVall=np.array(list(map(oneVsAll, prediction)))
      #print(len(np.unique(actual)))
      pppp = pd.DataFrame(oneVall)
      print("onevs : ",pppp.groupby(0)[0].count())
      #print(len(np.unique(oneVall)))
      #print("accurcy---",i,"  ",accuracy_score(actual,oneVall)*100)
      m[0].append(accuracy_score(actual,oneVall)*100)
      m[1].append(precision_score(actual,oneVall))
      m[2].append(recall_score(actual,oneVall))
      m[3].append(f1_score(actual,oneVall))
      m[4].append(roc_auc_score(actual,oneVall))
      
    
    i_index = m[0].index(max(m[0]))  
    #print("Index--------",i_index)
    myScore.append(m[0][i_index])
    myScore.append(m[1][i_index])
    myScore.append(m[2][i_index])
    myScore.append(m[3][i_index])
    myScore.append(m[4][i_index])

    last = 0
    #print("accurcy max---",myScore[0])
  return myScore[0]
  


def prec_score(estimator,x,y):
  return myScore[1]

def rec_score(estimator,x,y):
  return myScore[2]

def f1_s_score(estimator,x,y):
  return myScore[3]

def roc_score(estimator,x,y):
  return myScore[4]




loadData()


#addOtherLabel()
removeDuplicate()
#balanceClass()
#print(df_train.groupby('label')['label'].count())
splitData()

X_train, X_test = one_hot_encode(X_train,X_test)



scalers_to_test = [MinMaxScaler(),None]#
 
params = {'scaler': scalers_to_test,
          'reduce_dim': [None,PCA(n_components=5)],
          'regressor__n_clusters': [2],
          'regressor__random_state':[0]}
          
"""
....................................or (both result same)
params = [{'scaler': scalers_to_test,
          'reduce_dim': [PCA()],
          'reduce_dim__n_components': [5,20,40,55],
          'regressor__n_clusters': [2,5],
          'regressor__random_state':[0,2000,10000,50000,100000]}
          ,
          {
          'scaler': scalers_to_test,
          'reduce_dim': [None],
          'regressor__n_clusters': [2,5],
          'regressor__random_state':[0,2000,10000,50000,100000]    
          }]

"""
pipe = Pipeline(steps=[('scaler', StandardScaler()),('reduce_dim', PCA()),('regressor', KMeans())],verbose=2)


scoring = {'accuracy': make_scorer(acc_score,greater_is_better=True,method='acc'),
           'prec': prec_score,
           'rec': rec_score,
           'f1_s': f1_s_score,
            'roc': roc_score}    


"""cv= [(slice(None), slice(None))]# for non crossvalidation
cv1 = ShuffleSplit(test_size=0.20, n_splits=1, random_state=0)#only use 1 cvset as 0.2 of training set
"""
grid = GridSearchCV(pipe, cv=cvCount, scoring=scoring,param_grid= params,return_train_score=trainScore,refit=False,n_jobs=1, verbose=2)#
grid.fit(X_train,Y_train)

cvResult = pd.DataFrame(grid.cv_results_)
cvResult
#rmtree(cachedir)

In [ ]:
print(".......................................................................GMM.............")
trainScore = True
cvCount = 5

last=1 #test=0
i_index=-1
myScore=[]
iCount=0
fitNum=0


def acc_score(actual,prediction,method):
  global last
  global i_index
  global myScore
  global iCount
  global fitNum
 

  m=[[],[],[],[],[]]
  cls = -1
  
  
  def oneVsAll(num):
    if (num ==cls):
      return 0
    else:
      return 1

  if (last == 0 and trainScore):
     #trainset
    print("in train")
    
    myScore.clear()
    cls = i_index
    #print("Index--------",i_index)
    oneVtest =np.array(list(map(oneVsAll, prediction)))
    myScore.append(accuracy_score(actual,oneVtest)*100)
    myScore.append(precision_score(actual,oneVtest))
    myScore.append(recall_score(actual,oneVtest))
    myScore.append(f1_score(actual,oneVtest))
    myScore.append(roc_auc_score(actual,oneVtest))

    last = 1
    
  else:
   #testset
    print("in test")
    lena = len(np.unique(actual))
    lenp = len(np.unique(prediction))
    if (lena>lenp):
      n = lena
    else:
      n= lenp
    if (iCount%cvCount == 0):
      
      fitNum =fitNum + 1
      print("-----------------------------------------------------------",fitNum,"--------------------------------------------------------")

    myScore.clear()
    
    for i in range(0, n):
      cls=i
      oneVall=np.array(list(map(oneVsAll, prediction)))
      #print(len(np.unique(actual)))
      #pppp = pd.DataFrame(oneVall)
      #print("onevs : ",pppp.groupby(0)[0].count())
      #print(len(np.unique(oneVall)))
      #print("accurcy---",i,"  ",accuracy_score(actual,oneVall)*100)
      m[0].append(accuracy_score(actual,oneVall)*100)
      m[1].append(precision_score(actual,oneVall))
      m[2].append(recall_score(actual,oneVall))
      m[3].append(f1_score(actual,oneVall))
      m[4].append(roc_auc_score(actual,oneVall))
      
    
    i_index = m[0].index(max(m[0]))  
    #print("Index--------",i_index)
    myScore.append(m[0][i_index])
    myScore.append(m[1][i_index])
    myScore.append(m[2][i_index])
    myScore.append(m[3][i_index])
    myScore.append(m[4][i_index])

    last = 0
    #print("accurcy max---",myScore[0])
  return myScore[0]
  


def prec_score(estimator,x,y):
  return myScore[1]

def rec_score(estimator,x,y):
  return myScore[2]

def f1_s_score(estimator,x,y):
  return myScore[3]

def roc_score(estimator,x,y):
  return myScore[4]




loadData()

#print(df_train.groupby('label')['label'].count())
#addOtherLabel()
removeDuplicate()
#balanceClass()
splitData()

X_train, X_test = one_hot_encode(X_train,X_test)


cv_types = [ 'tied']
scalers_to_test = [StandardScaler(), MinMaxScaler(),None]
 
params = {'scaler': scalers_to_test,
          'reduce_dim': [None,PCA(n_components=5),PCA(n_components=5)],
          'regressor__n_components': [10],
          'regressor__covariance_type': cv_types,
          'regressor__random_state':[0]}
          
"""
....................................or (both result same in reduce__dim)
params = [{'scaler': scalers_to_test,
          'reduce_dim': [PCA()],
          'reduce_dim__n_components': [5,20,40,55],
          'regressor__n_clusters': [2,5],
          'regressor__random_state':[0,2000,10000,50000,100000]}
          ,
          {
          'scaler': scalers_to_test,
          'reduce_dim': [None],
          'regressor__n_clusters': [2,5],
          'regressor__random_state':[0,2000,10000,50000,100000]    
          }]

"""
pipe = Pipeline(steps=[('scaler', StandardScaler()),('reduce_dim', PCA()),('regressor', GaussianMixture())],verbose=2)


scoring = {'accuracy': make_scorer(acc_score,greater_is_better=True,method='acc'),
           'prec': prec_score,
           'rec': rec_score,
           'f1_s': f1_s_score,
            'roc': roc_score}          



grid = GridSearchCV(pipe, cv=cvCount, scoring=scoring,param_grid= params,return_train_score=trainScore,refit='accuracy',n_jobs=1, verbose=2)#

grid.fit(X_train,Y_train)

cvResult = pd.DataFrame(grid.cv_results_)

#rmtree(cachedir)

In [ ]:
cvResult.sort_values(['rank_test_accuracy'])

In [ ]:
cc = ['param_reduce_dim','param_regressor__covariance_type', 'param_regressor__n_components','param_regressor__random_state','param_scaler','params','mean_test_accuracy','rank_test_accuracy','mean_test_prec','rank_test_prec','mean_test_rec','rank_test_rec','mean_test_f1_s','rank_test_f1_s','mean_test_roc','rank_test_roc']
s = cvResult[cc]
s.sort_values(['rank_test_accuracy'])

In [ ]:
nu = 10

print(grid.cv_results_['param_reduce_dim'][nu])
print(grid.cv_results_['params'][nu])


In [ ]:
print("GMM \n\n")
loadData()
#df_train.columns
#print(df_train.groupby('attack_cat')['attack_cat'].count())

#print(df_train.groupby('label')['label'].count())
#addOtherLabel()
removeDuplicate()
#balanceClass()
splitData()


X_train, X_test = one_hot_encode(X_train,X_test)

X_train, X_test = scaling(X_train,X_test,MinMaxScaler())
X_train,X_test = runPCA(X_train,X_test,5)

gmm= GaussianMixture(n_components=10,random_state=0,covariance_type='full')
gmm.fit(X_train)

In [ ]:
predictedTrain = gmm.predict(X_train)
predictedTest= gmm.predict(X_test)

In [ ]:
oneVsAllScore(10,predictedTrain, predictedTest)